In this kernel I try CNN to distinguish between cats and dogs.


> ## 1. Import Libraries

In [ ]:
# data load
import zipfile
import cv2
from keras.preprocessing.image import ImageDataGenerator, load_img

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# making pictures
import matplotlib.pyplot as plt

# CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

import os

> ## 2. Prepare Train Data

In [ ]:
print(os.listdir("../input"))
print(os.listdir("../input/dogs-vs-cats"))

path = "../input/dogs-vs-cats/"

Let's unzip the data first:

In [ ]:
with zipfile.ZipFile(path + "train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile(path + "test1.zip","r") as z:
    z.extractall(".")
    
# saved as output to "/kaggle/working/train" and "/kaggle/working/test1"

In [ ]:
path_train = "/kaggle/working/train/"
filenames = os.listdir(path_train)
is_dog = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        is_dog.append(1)
    else:
        is_dog.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'is_dog': is_dog
})

To verify that we read the labels correctly:

In [ ]:
df.head()

### 2.1 Brief Look at the Data

We have (about) equal number of cats and dogs in the train sample:

In [ ]:
df['is_dog'].value_counts().plot.bar();

Sample picture:

In [ ]:
image = load_img(path_train + random.choice(filenames))
plt.imshow(image);

### 2.2 Images to Arrays****

Let's use cv2 library to read images into arrays:

In [ ]:
X = []
y = []
convert = lambda category : int(category == 'dog')
def read_train_data(path):
    for pic in os.listdir(path):
        category = pic.split(".")[0]
        is_dog = convert(category)
        img_array = cv2.imread(os.path.join(path,pic), cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(is_dog)

read_train_data(path_train)

In [ ]:
X = np.array(X).reshape(-1, 80,80, 1)
y = np.array(y)

Check if the shapes are matching:

In [ ]:
print(X.shape)
print(y.shape)

Gray scale pictures often work better than coloured, so let's normalize them:

In [ ]:
X = X/255.0  # from 0-255 to 0-1 scale

> ## 3. Train the Model

Basic model from https://www.kaggle.com/ruchibahl18/cats-vs-dogs-basic-cnn-tutorial/notebook

1. First we will add a Conv2D layer with 64 nodes and kernel size of (3,3). You can also experiment with different values here like 32, 128 etc. Also we have to specify input shape which is your X shape. Activation we will take 'relu' for now however there are many others to experiment with.
2. Now after every Conv layer we always do max pooling so we will add max pooling layer with a size of (2,2)
3. We will repeat this combination again because come on 2 is better than one. Haha. We you can also add 3 or more convolution layers but keep in mind the more layers you add more time it will take to train.
4. But we don't have much time so we will add a flatten layer now. As we have to feed our data to Dense layer later.
5. We will now add a Dense layer of 64 nodes. Note for all these layers we are using activation as 'relu' because I found results better with this. You can skip specifying activation but this might make a model a conveniently linear which might not work for us.
6. In the end for getting our result we will add final Dense layer . Activation can be sigmoid or softmax (if you need probability use sigmoid else use softmax). Here I have used sigmoid.
7. Finally we will compile the model

In [ ]:
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

> ## 4. Make Predictions
### 4.1 Prepare test data

In [ ]:
path_test = "/kaggle/working/test1/"

X_test = []
id_line = []
def read_test1_data(path):
    for pic in os.listdir(path):
        id_line.append(pic.split(".")[0])
        img_array = cv2.imread(os.path.join(path,pic), cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
read_test1_data(path_test)
X_test = np.array(X_test).reshape(-1,80,80,1)
X_test = X_test/255

### 4.2 Predict, Generate Submission

In [ ]:
predictions = model.predict(X_test)
predicted_val = [int(round(p[0])) for p in predictions]
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})
submission_df.to_csv("submission.csv", index=False)

In [ ]:
for pic in os.listdir(path_test):
    os.remove(path_test + pic)
for pic in os.listdir(path_train):
    os.remove(path_train + pic)